# Loading and combining the data sources

In [3]:
import pytz
import numpy as np
import pandas as pd
from pathlib import Path

import numba

#### Define functions for loading and combining separate data sources

In [5]:
# TODO: find out whether the datetime object of the power production should be associated to StartTimeUTC or EndTimeUTC: 
### both for the actual power and for the day ahead prices
def load_actual_wind_power(DATA_DIR):
    print("Loading actual wind power production...")

    # Read csv-file
    actual_wind_power                   = pd.read_csv(DATA_DIR / 'raw/Actual wind power.csv', sep=';')
    # Parse datetime by combining date and hour information
    actual_wind_power['StartTimeUTC']   = pd.to_datetime(actual_wind_power['Date'] + ' ' + actual_wind_power['Time'], format='mixed')
    # Add timezone to datetime element
    actual_wind_power['StartTimeUTC']   = actual_wind_power.StartTimeUTC.dt.tz_localize(pytz.UTC) 

    # Assume that the timestamp is from DK (since it starts at hour 0 of 2021 which the data from the other files does in DK time.
    # For this reason we adjust the timeseries and express everything in terms of UTC timestamps 
    actual_wind_power['StartTimeUTC']   = actual_wind_power.StartTimeUTC - pd.to_timedelta('2 hours')

    # Get rid of redundant columns
    actual_wind_power = actual_wind_power.drop(columns=['Date', 'Time'])
    return actual_wind_power

def load_balancing_prices(DATA_DIR):
    print("Loading balancing prices...")

    # Load information about up- and down-regulation prices for both 2021 and 2022
    for year_idx, year in enumerate([2021, 2022]):
        for i, filename in enumerate([f'Down-regulation price_{year}.csv', f'Up-regulation price_{year}.csv']):
            # Determine filetype
            price_type          = filename.split('-')[0]
            price_column_name   = 'Up-regulating' if price_type == 'Up' else 'Down-regulation' 
            price_column_name   = f'"{price_column_name} price in the Balancing energy market"""'
            
            # Read csv-file as temporary dataframe
            df_price_ = pd.read_csv(DATA_DIR / f'raw/{filename}', sep=',"', engine='python')
        
            # Handle encoding with quotation marks
            df_price_['StartTimeUTC']   = pd.to_datetime(df_price_['"Start time UTC'].str.strip('"')).dt.tz_localize(pytz.UTC)
            df_price_['EndTimeUTC']     = pd.to_datetime(df_price_['"End time UTC""'].str.strip('"')).dt.tz_localize(pytz.UTC)

            # Change datatype of prices from string to float
            df_price_[f'BalancingMarketPrice_{price_type}Reg'] = df_price_[price_column_name].str.strip('"')
            df_price_[f'BalancingMarketPrice_{price_type}Reg'] = df_price_[f'BalancingMarketPrice_{price_type}Reg'].astype(float)
        
            # Restrict data to relevant information - Danish timezone is implicitly contained in UTC timestamp
            df_price_ = df_price_[['StartTimeUTC', 'EndTimeUTC', f'BalancingMarketPrice_{price_type}Reg']]


            # Combine dataframes for both years 
            prices_ = df_price_ if i == 0 else prices_.merge(df_price_, on=['StartTimeUTC', 'EndTimeUTC'], how='outer')
            
        # Merge prices from year with currently stored price information into combined dataframe
        balancing_prices = prices_ if year_idx == 0 else pd.concat([balancing_prices, prices_], axis=0).reset_index(drop=True)
        
    return balancing_prices

# TODO: find out whether the datetime object of the power production should be associated to StartTimeUTC or EndTimeUTC: 
### both for the actual power and for the day ahead prices
def load_day_ahead_prices(DATA_DIR):
    print("Loading day-ahead prices...")

    # Read day ahead prices from excel sheet
    day_ahead_prices = pd.read_excel(DATA_DIR / 'raw/Day-ahead price.xlsx')

    # Represent time as datetime object
    day_ahead_prices['StartTimeUTC'] = pd.to_datetime(day_ahead_prices['HourUTC']).dt.tz_localize(pytz.UTC)

    # Get rid of redundant information
    day_ahead_prices = day_ahead_prices[['StartTimeUTC', 'PriceArea', 'SpotPriceDKK', 'SpotPriceEUR']]
    return day_ahead_prices


def combine_and_save_data_sources(DATA_DIR: Path, SAVE_DIR: Path):
    # Load and do initial processing of data files
    actual_wind_power   = load_actual_wind_power(DATA_DIR)
    balancing_prices    = load_balancing_prices(DATA_DIR)
    day_ahead_prices    = load_day_ahead_prices(DATA_DIR)

    # Merge data sources based on temporal information and pricearea
    dataset = actual_wind_power.merge(day_ahead_prices, on='StartTimeUTC', how='left').merge(balancing_prices, on='StartTimeUTC')

    # TODO: consider what to do with summer/wintertime hours - here we take the mean
    dataset = dataset.groupby(by=['StartTimeUTC', 'EndTimeUTC', 'PriceArea']).mean().reset_index()

    # Save loaded and combined data files
    dataset.to_csv(SAVE_DIR / 'combined.csv')
    actual_wind_power.to_csv(SAVE_DIR / 'actual_wind_power.csv')
    balancing_prices.to_csv(SAVE_DIR / 'balancing_prices.csv')
    day_ahead_prices.to_csv(SAVE_DIR / 'day_ahead_prices.csv')
    
    return dataset, actual_wind_power, balancing_prices, day_ahead_prices    

#### Load and save data files

In [8]:
# Set path to data and save folder
DATA_DIR = Path('../../../data/assignment1')
SAVE_DIR = Path('../../../data/assignment1/processed')

# Load and combine data sources
dataset, _, _, _ = combine_and_save_data_sources(DATA_DIR, SAVE_DIR)

Loading actual wind power production...
Loading balancing prices...
Loading day-ahead prices...


In [15]:
print(f"NaN values occuring in the dataset? {dataset.isna().any().any()}")

NaN values occuring in the dataset? False


#### Exploration and loading of weather data

In [4]:
import json
import zipfile
from tqdm import tqdm

In [5]:
# zip file handler  
zip = zipfile.ZipFile('C:/Users/alber/Desktop/DTU/3_HCAI/46765/ml-energy-systems/data/assignment1/raw/Climate data_2021.zip')
f = zip.open('2021-01-01.txt', 'r')

# Do initial investigation of relevant attributes by loading a single sample file
information_list = []
for line in f:
    information_list.append(json.loads(line))

# Ex
weather_attributes = pd.Series([information['properties']['parameterId'] for information in information_list]).unique()
print(f"Unique weather attributes: \n{weather_attributes}")

municipalities = pd.Series([information['properties']['municipalityName'] for information in information_list]).unique()
print(f"\nUnique municipalities: \n{municipalities}")

Unique weather attributes: 
['no_ice_days' 'temp_grass' 'leaf_moisture' 'mean_temp' 'mean_wind_speed'
 'max_temp_w_date' 'mean_cloud_cover' 'temp_soil_30' 'no_summer_days'
 'temp_soil_10' 'mean_daily_max_temp' 'no_lightning_strikes'
 'max_wind_speed_10min' 'bright_sunshine' 'no_tropical_nights'
 'no_cold_days' 'no_days_acc_precip_1' 'min_temp' 'drought_index'
 'mean_radiation' 'no_days_acc_precip_01' 'acc_heating_degree_days_17'
 'max_wind_speed_3sec' 'vapour_pressure_deficit_mean'
 'pot_evaporation_makkink' 'mean_pressure' 'mean_daily_min_temp'
 'mean_relative_hum' 'acc_precip' 'mean_wind_dir' 'no_days_acc_precip_10'
 'no_frost_days' 'max_precip_30m' 'snow_depth']

Unique municipalities: 
['Furesø' 'Struer' 'Egedal' 'Faxe' 'Fredericia' 'Frederikssund' 'Hvidovre'
 'Lemvig' 'Kalundborg' 'Vesthimmerlands' 'Lejre' 'Haderslev' 'Syddjurs'
 'Thisted' 'Allerød' 'Odense' 'Solrød' 'Svendborg' 'Hørsholm'
 'Lyngby-Taarbæk' 'Gribskov' 'Ikast-Brande' 'Vordingborg' 'Stevns' 'Samsø'
 'Sønderborg' 'Ro

In [35]:
def process_climate_data(year, DATA_DIR, SAVE_DIR):
    
    # Open zip folder
    zip = zipfile.ZipFile(DATA_DIR / f'raw/Climate data_{year}.zip')

    # Extract information from all files within the zip-folder
    weather_information = []
    for i, filename in enumerate(tqdm(zip.namelist(), desc=f'Processing {year}')):
        # Open file
        f = zip.open(filename, 'r')
        for line in f:
            # Read each entry individually
            information = json.loads(line)
            
            # Restrict the extracted information to temporal, location and weather-related information only 
            new_observation = [
                information['properties']['from'], 
                information['properties']['to'], 
                information['properties']['parameterId'], 
                information['properties']['value'], 
                information['geometry']['coordinates'][0], 
                information['geometry']['coordinates'][1], 
                information['properties']['municipalityName']
            ]
            weather_information.append(new_observation)

    # Create dataframe of weather from the given year
    column_names        = ['StartTimeUTC', 'EndTimeUTC', 'WeatherAttribute', 'Value', 'Longitude', 'Latitude', 'Municipality'] 
    weather_information = pd.DataFrame(weather_information, columns=column_names)
    
    # Save information
    weather_information.to_csv(SAVE_DIR / f'weather_information_{year}.csv')

In [36]:
# Set path to data and save folder
DATA_DIR = Path('../../../data/assignment1')
SAVE_DIR = Path('../../../data/assignment1/processed')

process_climate_data(2021, DATA_DIR, SAVE_DIR)
process_climate_data(2022, DATA_DIR, SAVE_DIR)

Processing 2022: 100%|██████████| 365/365 [02:44<00:00,  2.22it/s]
